In [ ]:
import asyncio
import csv
import json
import aiohttp
from playwright.async_api import async_playwright


async def scrape_company_details(url):
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            page = await browser.new_page()
            await page.goto(url)

            await page.wait_for_selector('.company-details')

            company_name = await page.inner_text('.company-details .name')
            company_description = await page.inner_text('.company-details .description')
            company_rating = await page.inner_text('.company-details .rating')
            return {
                'url': url,
                'name': company_name,
                'description': company_description,
                'rating': company_rating
            }
    except Exception as e:
        print(f"Error scraping URL: {url}")
        print(str(e))

    return None


async def scrape_g2crowd_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.ensure_future(scrape_company_details(url))
            tasks.append(task)

        results = await asyncio.gather(*tasks)

        return results


def save_results_to_json(results, output_file):
    with open(output_file, 'w') as file:
        json.dump(results, file, indent=4)


def save_results_to_csv(results, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)


async def main():
    url_file = 'g2crowd_urls.csv'
    output_file = 'scraped_data.json'

    with open(url_file, 'r') as file:
        reader = csv.reader(file)
        urls = [row[0] for row in reader]

    results = await scrape_g2crowd_urls(urls)
    save_results_to_json(results, output_file)


if __name__ == '__main__':
    asyncio.run(main())
